In [ ]:
import os
SOFTWARE_DIR = '/home/twongjirad/working/larbys/lartpc_mlreco3d/'
DATA_DIR = os.environ.get('DATA_DIR')
print(SOFTWARE_DIR)
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
# set software directory
sys.path.insert(0, SOFTWARE_DIR)

In [ ]:
import numpy as np
import yaml
import torch
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=False)

from mlreco.visualization import scatter_points, plotly_layout3d
from mlreco.visualization.gnn import scatter_clusters, network_topology, network_schematic
from mlreco.utils.ppn import uresnet_ppn_type_point_selector
from mlreco.utils.cluster.dense_cluster import fit_predict_np, gaussian_kernel
from mlreco.main_funcs import process_config, prepare
from mlreco.utils.gnn.cluster import get_cluster_label
from mlreco.utils.deghosting import adapt_labels_numpy as adapt_labels
from mlreco.visualization.gnn import network_topology

from larcv import larcv

In [ ]:
cfg=yaml.load(open('%s/config/train_ubmlreco_uresnet_ppn.cfg' % (SOFTWARE_DIR), 'r').read(),Loader=yaml.Loader)
# pre-process configuration (checks + certain non-specified default settings)
process_config(cfg)
# prepare function configures necessary "handlers"
hs=prepare(cfg)

In [ ]:
# Call forward to run the net, store the output in "res"
data, output = hs.trainer.forward(hs.data_io_iter)

In [ ]:
entry = 0
print(data.keys())
print(output.keys())

In [ ]:
input_data = data['input_data'][entry]
segment_label = data['segment_label'][entry][:, -1]

ghost_mask = output['ghost'][entry].argmax(axis=1) == 0
segment_pred = output['segmentation'][entry].argmax(axis=1)

In [ ]:
trace = []

edep = input_data[segment_label < 5]

trace+= scatter_points(input_data[segment_label < 5],
                       markersize=0.5,
                       #color=segment_label[segment_label < 5], 
                       color="rgb(255,0,255,0.2)", 
                       cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'True semantic labels (true no-ghost mask)'

trace+= scatter_points(input_data[ghost_mask],
                       markersize=2,color=segment_pred[ghost_mask], 
                       cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Predicted semantic labels (predicted no-ghost mask)'

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)

In [ ]:
# Semantic error and correctness

trace = []

# pick out true deposits only
edep = input_data[segment_label < 5]
seg_label = segment_label[ segment_label<5 ]
seg_pred = segment_pred[segment_label<5]
correct = seg_pred==seg_label
wrong   = seg_pred!=seg_label
print("correct: ",correct.sum())
print("incorrect: ",wrong.sum())

trace+= scatter_points(edep[ correct ],
                       markersize=0.5,
                       #color=segment_label[segment_label < 5], 
                       color="rgb(0,0,255,0.2)", 
                       cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Correct prediction'

trace+= scatter_points(edep[ wrong ],
                       markersize=0.5,
                       #color=segment_label[segment_label < 5], 
                       color="rgb(255,0,0,0.2)", 
                       cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Wrong prediction'

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)


In [ ]:
# Track predictions only


trace = []

# pick out true deposits only
seg_label_track = (segment_label>1) & (segment_label<5)
print("num track labels: ",seg_label_track.sum())
seg_pred_track = (segment_pred>1) & (segment_pred<5) & ghost_mask
print("num track predct: ",segment_pred.sum())

trace+= scatter_points(input_data[ seg_label_track ],
                       markersize=0.5,
                       #color=segment_label[segment_label < 5], 
                       color="rgb(255,0,255,0.2)", 
                       cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Track label'

trace+= scatter_points(input_data[ seg_pred_track ],
                       markersize=0.5,
                       #color=segment_label[segment_label < 5], 
                       color="rgb(0,0,255,0.7)", 
                       cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Track prediction'


fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)


In [ ]:
points = output['points'][entry]
print("points: ",points.shape)
# 0,1,2: pixel prediction
# 3,4,5,6,7: class score
# 8,9: final score
# print(points[:20,:])

In [ ]:
# RAW PPN INFORMATION FROM "POINTS" output

import scipy.special
points = output['points'][entry]
ppn_layers_logits_v = output['ppn_layers'][entry]
print("input: ",input_data.shape)
print("points: ",points.shape)
for x in ppn_layers_logits_v:
    print("ppn_layers_logits: ",x.shape)
pred_shift  = points[:,0:3]
ppn_kp      = input_data[:,1:4] + pred_shift
ppn_score   = points[:,-1]
type_logits = points[:,3:3+5]
type_score  = scipy.special.softmax(type_logits,axis=1)

input_noghost  = input_data[ghost_mask]
ppn_noghost    = ppn_score[ghost_mask]
ppn_kp_noghost = ppn_kp[ghost_mask]
type_score    = type_score[ghost_mask]
type_class    = np.argmax(type_score,axis=1)
print("type classes: ",np.unique(type_class))

trace = []

trace+= scatter_points(input_noghost,
                       markersize=0.5,color=ppn_noghost, 
                       cmin=0, cmax=1.0, 
                       colorscale='Viridis')
trace[-1].name = 'PPN Last Layer Score'

trace+= scatter_points(ppn_kp_noghost[ppn_noghost>0.7],
                       markersize=3,color=type_class[ppn_noghost>0.7], 
                       cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'PPN Max Type'

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)


In [ ]:
from mlreco.utils.ppn import uresnet_ppn_type_point_selector
ppn = uresnet_ppn_type_point_selector(data['input_data'][entry], output, entry=entry,
                                      score_threshold=0.8, type_threshold=2,
                                      enforce_type=False,
                                      scale_index=-1)

print(ppn.shape)

ppn_voxels = ppn[:, 1:4]
ppn_score = ppn[:, 5]
ppn_type = ppn[:, 12]
ppn_endpoints = np.argmax(ppn[:, 13:], axis=1)


In [ ]:
delta_label = 3
is_not_delta = ppn[:, 7 + delta_label] < 0.5
ppn_voxels = ppn_voxels[is_not_delta]
ppn_score = ppn_score[is_not_delta]
ppn_type = ppn_type[is_not_delta]
ppn_endpoints = ppn_endpoints[is_not_delta]

In [ ]:
trace = []

PLOT_TRUEPTS = False

trace+= scatter_points(input_data[ghost_mask],markersize=0.5,
                       color=segment_pred[ghost_mask], 
                       cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Predicted semantic labels (predicted no-ghost mask)'

trace += scatter_points(ppn_voxels, markersize=5, color=ppn_type, cmin=0, cmax=10, 
                        colorscale=plotly.colors.qualitative.D3, hovertext=ppn_score)
trace[-1].name = "PPN predictions (w/ type prediction)"

trace += scatter_points(ppn_voxels[ppn_type == 1], markersize=5, color=ppn_endpoints[ppn_type == 1], 
                        cmin=0, cmax=10, 
                        colorscale=plotly.colors.qualitative.D3, 
                        hovertext=ppn_endpoints)
trace[-1].name = "PPN predictions (start/end)"

if PLOT_TRUEPTS:
    trace += scatter_points(data['particles_label'][entry], markersize=2, 
                       color=data['particles_label'][entry][:, 4], cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
    trace[-1].name = "True point labels"

    trace += scatter_points(data['particles_label'][entry], markersize=2, 
                       color=data['particles_label'][entry][:, 6], cmin=0, cmax=10, 
                       colorscale=plotly.colors.qualitative.D3)
    trace[-1].name = "True point labels (start/end)"

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)
